In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\camilo\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact',
       'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg',
       'koi_srad', 'ra', 'dec', 'koi_kepmag']]

In [6]:
y = df[["koi_disposition"]]

In [7]:
y = y.values.reshape(-1,)
y.shape

(6991,)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.20)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
1395,0,0,0,0,28.505722,132.920750,0.6950,6.3240,755.8,2.55,555,22.43,20.8,1,5712,4.534,0.870,292.23242,38.892220,15.778
4336,1,0,0,0,168.771180,233.313800,36.9452,2.2630,2045.0,2825.04,251,0.93,5.5,1,4936,4.597,0.719,296.40408,39.169811,15.941
2668,0,0,0,0,85.316200,295.315880,0.9720,2.7441,1902.6,6.34,423,7.57,32.6,3,6046,4.486,0.972,296.47977,49.937679,14.669
5807,0,1,0,0,3.495458,132.083941,0.9650,6.9796,114620.0,84.49,2051,4186.71,724.0,1,8529,4.249,1.558,294.07187,38.426922,13.444
3813,0,1,0,0,30.552870,143.348399,1.1340,5.3367,63521.0,63.57,685,51.90,510.8,1,6441,4.367,1.165,290.96396,38.469501,16.245


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model



In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=3, activation='softmax'))

In [15]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [18]:
model.fit(X_train_scaled,
         y_train_categorical,
         epochs=100,
         shuffle=True,
         verbose=2)

Train on 5592 samples
Epoch 1/100
5592/5592 - 0s - loss: 0.3271 - accuracy: 0.8362
Epoch 2/100
5592/5592 - 0s - loss: 0.3254 - accuracy: 0.8421
Epoch 3/100
5592/5592 - 0s - loss: 0.3250 - accuracy: 0.8358
Epoch 4/100
5592/5592 - 0s - loss: 0.3247 - accuracy: 0.8387
Epoch 5/100
5592/5592 - 0s - loss: 0.3238 - accuracy: 0.8405
Epoch 6/100
5592/5592 - 0s - loss: 0.3232 - accuracy: 0.8378
Epoch 7/100
5592/5592 - 0s - loss: 0.3212 - accuracy: 0.8419
Epoch 8/100
5592/5592 - 0s - loss: 0.3215 - accuracy: 0.8437
Epoch 9/100
5592/5592 - 0s - loss: 0.3201 - accuracy: 0.8459
Epoch 10/100
5592/5592 - 0s - loss: 0.3205 - accuracy: 0.8417
Epoch 11/100
5592/5592 - 0s - loss: 0.3190 - accuracy: 0.8430
Epoch 12/100
5592/5592 - 0s - loss: 0.3184 - accuracy: 0.8437
Epoch 13/100
5592/5592 - 0s - loss: 0.3174 - accuracy: 0.8414
Epoch 14/100
5592/5592 - 0s - loss: 0.3166 - accuracy: 0.8439
Epoch 15/100
5592/5592 - 0s - loss: 0.3160 - accuracy: 0.8457
Epoch 16/100
5592/5592 - 0s - loss: 0.3153 - accuracy: 0.

In [20]:
print(f"Training Data Score: {model.evaluate(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {model.evaluate(X_test_scaled, y_test_categorical)}")

5592/5592 [==============================] - 0s 34us/sample - loss: 0.2717 - accuracy: 0.8704
Training Data Score: [0.27171758870369034, 0.8703505]
1399/1399 [==============================] - 0s 29us/sample - loss: 0.2900 - accuracy: 0.8635
Testing Data Score: [0.2899584426378846, 0.8634739]


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [23]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

# Save the Model

In [21]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)

TypeError: can't pickle _thread.RLock objects